# Preprocessing of wind turbine data

Cells in this notebook can be run independently provided the first two ("Imports" and "Preprocessing function...") are run first.

1. [Imports.](#Cell1)
1. [Preprocessing function, good for training data as well as testing data.](#Cell2)
1. [Preprocess the training data set, output a pickle file.](#Cell3)
1. [Preprocess the training data set in Feather format for notebooks running R kernel.](#Cell4)
1. [Preprocess the testing data set, output a pickle file.](#Cell5)
1. [Preprocess the testing data set in Feather format for notebooks running R kernel.](#Cell6)
1. [Preprocess the solution file (the testing data set with TARGETVAR included). Output in pickle and Feather.](#Cell7)

<a id='Cell1'></a>

In [19]:
'''
Imports.
'''
import feather
import pandas as pd
import numpy as np
from time import strftime
import datetime
from IPython.display import display_html

<a id='Cell2'></a>

In [17]:
'''
Preprocessing function, good for training data as well as testing data.
'''
def preProcess(fname, Nroll=3, inclusive=False, keepID=False):
    '''
    Read in the wind turbine data, store in a Pandas dataframe, and rearrange.

    1.  Records in the input csv file are assumed to have the following form:
            ID, ZONEID, TIMESTAMP, TARGETVAR, U10, V10, U100, V100
        where:
            ID =         Unique ID for each observation
            ZONEID =     Zone/Turbine ID, a number between 1 and 10
            TIMESTAMP =  Time of observation, in the format "YYYYMMDD h:mm" or "YYYYMMDD hh:mm"
            TARGETVAR =  Wind turbine output (only present in the training data set)
            U10 =        Zonal Wind Vector at 10 m
            V10 =        Meridional Wind Vector at 10 m
            U100 =       Zonal Wind vector at 100 m
            V100 =       Meridional Wind vector at 100 m
    
    2. If Nroll<=1, column names in the output data frame have the form:
            TARGETVAR U10 V10 U100 V100 YEAR DAYOFYEAR HOUR
       and TARGETVAR, U10, V10, U100, V100 each have a subindex 1 ... 10 indicating zone id.
       
       If Nroll>1, the structure is the same, but the U10, V10, U100, V100 columns are replaced
       by rolling means over Nroll measurements, and the names are U10_rmX, V10_rmX, U100_rmX, 
       and V100_rmX, where X equals Nroll.
       
       If Nroll>1 and inclusive==True, all rolling means from 1 to Nroll are kept in the 
       dataframe. If inclusive==False, only the rolling mean corresponding to Nroll is kept.
    '''

    # Read in the windmill data, parsing the third column as datetimes. Make a hierarchical
    # index out of the timestamp and zone id columns.  Then drop the measurement ID column.
    df0 = pd.read_csv(fname, header=0, parse_dates=[2], index_col=[2,1])
    if not keepID: 
        df0 = df0.drop('ID', 1)
        print('ID column dropped')
    else:
        print('ID column kept')
        
    # Unstack the inner level of the index. The index is thus reduced to the timestamp, and 
    # the zone ids become subcolumns within the existing columns. In other words, where in
    # df0 there are 10 rows per time stamp, one for each zone id, in df1 there is only 
    # 1 row, which contains the measurements made at the given time stamp in all the zone ids.
    df1 = df0.unstack()

    # Put the timestamp index back as a dataframe column and verify that all delta-times are one hour.
    df2              = df1.reset_index()
    dtimes           = df2["TIMESTAMP"].subtract(df2["TIMESTAMP"].shift(+1))
    Ntimepoints      = len(df2)
    Ndeltatimes1h    = dtimes[dtimes==np.timedelta64(1, 'h')].count()
    assert (Ntimepoints == Ndeltatimes1h+1), 'Delta times not all equal to 1 h!'
    print('Number of time points:                 {0}'.format(Ntimepoints))
    print('Number of delta-times equal to 1 hour: {0}'.format(Ndeltatimes1h))
    
    # Remove duplicate wind measurements (5 is duplicate with 4, and 8 with 7)
    nturbines         = 10
    wind_measurements = [1, 2, 3, 4, 6, 7, 9, 10]
    for i in range(1,nturbines+1):
        if i not in wind_measurements:
            df2 = df2.drop([("U10",i), ("V10",i), ("U100",i), ("V100",i)], axis=1)
    
    # Add rolling means of the wind measurements
    if Nroll>1:
        NrollSet = range(2, Nroll+1) if inclusive else [Nroll]
        for n in NrollSet:
            print('Computing rolling means over {0} measurements'.format(n))
            dname = "_rm"+str(n)
            for i in wind_measurements:
                df2[("U10"+dname,i)]  = df2[("U10",i)].rolling(n, min_periods=1).mean()
                df2[("U100"+dname,i)] = df2[("U100",i)].rolling(n, min_periods=1).mean()
                df2[("V10"+dname,i)]  = df2[("V10",i)].rolling(n, min_periods=1).mean()
                df2[("V100"+dname,i)] = df2[("V100",i)].rolling(n, min_periods=1).mean()
        if not inclusive: df2 = df2.drop(["U10", "U100", "V10", "V100"], axis=1, level=0)
    else:
        print('No rolling means computed')
    
    # Extract year, day of year, and hour from timestamp, then drop timestamp column
    df2["YEAR"]      = df2["TIMESTAMP"].map(lambda x: x.year)
    df2["DAYOFYEAR"] = df2["TIMESTAMP"].map(lambda x: x.timetuple().tm_yday)
    df2["HOUR"]      = df2["TIMESTAMP"].map(lambda x: x.hour)
    df2              = df2.drop('TIMESTAMP', axis=1, level=0)

    return df2

<a id='Cell3'></a>

In [9]:
'''
Preprocess the training data set, output a pickle file.
'''
# Load and preprocess data
train_csv = "data/Train_O4UPEyW.csv"
train_df  = preProcess(train_csv, Nroll=3, inclusive=False, keepID=False)
display_html(train_df.head())

# Output dataframe in pickle format
train_fname = "data/Train_pp_" + strftime("%Y_%m_%d_%H_%M_%S") + ".pkl"
try:
    train_df.to_pickle(train_fname)
    print('\nTraining data frame saved to {0}'.format(train_fname))
except:
    print('\nError saving training data frame to {0}'.format(train_fname))

ID column dropped
Number of time points:                 13871
Number of delta-times equal to 1 hour: 13870
Computing rolling means over 3 measurements

Dataframe train_df:
       TARGETVAR                                                              \
ZONEID         1         2         3         4         5         6         7   
0       0.000000  0.596273  0.425465  0.378229  0.273678  0.268101  0.000000   
1       0.054879  0.411180  0.363851  0.063012  0.086796  0.034542  0.014708   
2       0.110234  0.167243  0.297129  0.036704  0.006811  0.020621  0.035098   
3       0.165116  0.037326  0.235438  0.034342  0.018646  0.001867  0.045055   
4       0.156940  0.051206  0.120904  0.033554  0.034812  0.015174  0.050160   

                                ...   U100_rm3   V10_rm3  V100_rm3   U10_rm3  \
ZONEID         8    9        10 ...          9         9         9        10   
0       0.000000  0.0  0.594361 ...   2.516723 -1.495002 -2.078063  1.655790   
1       0.014954  0.0  0.5

<a id='Cell4'></a>

In [20]:
'''
Preprocess the training data set in Feather format for notebooks running R kernel.
'''
# Load and preprocess data
train_csv = "data/Train_O4UPEyW.csv"
train_df  = preProcess(train_csv, Nroll=4, inclusive=True, keepID=False)

# Rename dataframe columns
predictors  = [item[0]+'_'+str(item[1]) for item in train_df.columns.values if item[0]!='TARGETVAR']
predictors  = [predictor if predictor[-1]!='_' else predictor[:-1] for predictor in predictors]
targets     = [item[0]+'_'+str(item[1]) for item in train_df.columns.values if item[0]=='TARGETVAR']
train_df.columns = targets+predictors
display_html(train_df.head())

# Output data frame in feather format
train_fname = "data/Train_pp_" + strftime("%Y_%m_%d_%H_%M_%S") + ".feather"
try:
    feather.write_dataframe(train_df, train_fname)
    print('\nTraining data frame saved to {0}'.format(train_fname))
except:
    print('\nError saving training data frame to {0}'.format(train_fname))

ID column dropped
Number of time points:                 13871
Number of delta-times equal to 1 hour: 13870
Computing rolling means over 2 measurements
Computing rolling means over 3 measurements
Computing rolling means over 4 measurements


,TARGETVAR_1,TARGETVAR_2,TARGETVAR_3,TARGETVAR_4,TARGETVAR_5,TARGETVAR_6,TARGETVAR_7,TARGETVAR_8,TARGETVAR_9,TARGETVAR_10,...,U100_rm4_9,V10_rm4_9,V100_rm4_9,U10_rm4_10,U100_rm4_10,V10_rm4_10,V100_rm4_10,YEAR,DAYOFYEAR,HOUR
0,0.000000,0.596273,0.425465,0.378229,0.273678,0.268101,0.000000,0.000000,0.0,0.594361,...,2.516723,-1.495002,-2.078063,1.655790,2.024257,-4.649593,-5.992232,2012,1,1
1,0.054879,0.411180,0.363851,0.063012,0.086796,0.034542,0.014708,0.014954,0.0,0.569679,...,2.676916,-1.069321,-1.542488,1.917199,2.323699,-4.417269,-5.655159,2012,1,2
2,0.110234,0.167243,0.297129,0.036704,0.006811,0.020621,0.035098,0.071785,0.0,0.330539,...,2.795976,-0.667349,-1.033489,2.021010,2.430872,-4.025660,-5.120850,2012,1,3
3,0.165116,0.037326,0.235438,0.034342,0.018646,0.001867,0.045055,0.066796,0.0,0.211308,...,2.832439,-0.301223,-0.569234,2.035066,2.431390,-3.605516,-4.569095,2012,1,4
4,0.156940,0.051206,0.120904,0.033554,0.034812,0.015174,0.050160,0.034797,0.0,0.172140,...,2.843053,0.393942,0.318057,2.074797,2.449787,-2.837524,-3.565062,2012,1,5



Training data frame saved to data/Train_pp_2017_11_04_21_21_28.feather


<a id='Cell5'></a>

In [11]:
'''
Preprocess the testing data set, output a pickle file.
'''
test_csv  = "data/Test_uP7dymh.csv"

test_df = preProcess(test_csv, Nroll=3, inclusive=False, keepID=True)
display_html(test_df.head())

test_fname = "data/Test_pp_" + strftime("%Y_%m_%d_%H_%M_%S") + ".pkl"
try:
    test_df.to_pickle(test_fname)
    print('\nTesting data frame saved to {0}'.format(test_fname))
except:
    print('\nError saving testing data frame to {0}'.format(test_fname))

ID column kept
Number of time points:                 2929
Number of delta-times equal to 1 hour: 2928
Computing rolling means over 3 measurements

Dataframe test_df:
              ID                                                              \
ZONEID         1         2         3         4         5         6         7   
0       30010001  30010002  30010003  30010004  30010005  30010006  30010007   
1       30010101  30010102  30010103  30010104  30010105  30010106  30010107   
2       30010201  30010202  30010203  30010204  30010205  30010206  30010207   
3       30010301  30010302  30010303  30010304  30010305  30010306  30010307   
4       30010401  30010402  30010403  30010404  30010405  30010406  30010407   

                                     ...   U100_rm3   V10_rm3  V100_rm3  \
ZONEID         8         9        10 ...          9         9         9   
0       30010008  30010009  30010010 ...   1.473541 -3.757118 -5.748985   
1       30010108  30010109  30010110 ...   2.42

<a id='Cell6'></a>

In [22]:
'''
Preprocess the testing data set in Feather format for notebooks running R kernel.
'''
test_csv  = "data/Test_uP7dymh.csv"

test_df = preProcess(test_csv, Nroll=4, inclusive=True, keepID=True)

# Rename dataframe columns
predictors  = [item[0]+'_'+str(item[1]) for item in test_df.columns.values]
predictors  = [predictor if predictor[-1]!='_' else predictor[:-1] for predictor in predictors]
test_df.columns = predictors
display_html(test_df.head())
    
# Output data frame in feather format
test_fname = "data/Test_pp_" + strftime("%Y_%m_%d_%H_%M_%S") + ".feather"
try:
    feather.write_dataframe(test_df, test_fname)
    print('\nTesting data frame saved to {0}'.format(test_fname))
except:
    print('\nError saving testing data frame to {0}'.format(test_fname))

ID column kept
Number of time points:                 2929
Number of delta-times equal to 1 hour: 2928
Computing rolling means over 2 measurements
Computing rolling means over 3 measurements
Computing rolling means over 4 measurements


,ID_1,ID_2,ID_3,ID_4,ID_5,ID_6,ID_7,ID_8,ID_9,ID_10,...,U100_rm4_9,V10_rm4_9,V100_rm4_9,U10_rm4_10,U100_rm4_10,V10_rm4_10,V100_rm4_10,YEAR,DAYOFYEAR,HOUR
0,30010001,30010002,30010003,30010004,30010005,30010006,30010007,30010008,30010009,30010010,...,1.473541,-3.757118,-5.748985,-0.165117,0.320843,-2.658255,-4.277716,2013,213,0
1,30010101,30010102,30010103,30010104,30010105,30010106,30010107,30010108,30010109,30010110,...,2.425901,-3.923264,-5.798490,-0.257103,-0.014866,-3.223966,-4.800565,2013,213,1
2,30010201,30010202,30010203,30010204,30010205,30010206,30010207,30010208,30010209,30010210,...,2.899162,-4.043484,-5.885207,0.085075,0.330257,-3.693685,-5.259272,2013,213,2
3,30010301,30010302,30010303,30010304,30010305,30010306,30010307,30010308,30010309,30010310,...,2.975360,-4.085358,-5.894730,0.532278,0.866918,-4.032193,-5.622060,2013,213,3
4,30010401,30010402,30010403,30010404,30010405,30010406,30010407,30010408,30010409,30010410,...,3.362207,-4.201863,-5.941107,1.172676,1.591938,-4.641402,-6.259113,2013,213,4



Testing data frame saved to data/Test_pp_2017_11_04_21_29_23.feather


<a id='Cell7'></a>

In [21]:
'''
Preprocess the solution file. This is the testing data set with TARGETVAR included.
Preprocess it in the same way as the training data set, then output it in pickle and feather formats.
'''
# Load and preprocess data
solution_csv = "data/Solution.csv"
solution_df  = preProcess(solution_csv, Nroll=3, inclusive=False, keepID=False)
print('\nDataframe solution_df for pickle:\n')
display_html(solution_df.head())

# Output dataframe in pickle format
fname          = strftime("%Y_%m_%d_%H_%M_%S")
solution_fname = "data/Solution_pp_" + fname + ".pkl"
try:
    solution_df.to_pickle(solution_fname)
    print('\nSolution data frame saved to {0}'.format(solution_fname))
except:
    print('\nError saving solution data frame to {0}'.format(solution_fname))
    

# Rename dataframe columns for easier handling in R
fields  = [item[0]+'_'+str(item[1]) for item in solution_df.columns.values]
fields  = [field if field[-1]!='_' else field[:-1] for field in fields]
solution_df.columns = fields
print('\nDataframe solution_df for feather:\n')
display_html(solution_df.head())
    
# Output data frame in feather format
solution_fname = "data/Solution_pp_" + fname + ".feather"
try:
    feather.write_dataframe(solution_df, solution_fname)
    print('\nSolution data frame saved to {0}'.format(solution_fname))
except:
    print('\nError saving solution data frame to {0}'.format(solution_fname))

ID column dropped
Number of time points:                 2929
Number of delta-times equal to 1 hour: 2928
Computing rolling means over 3 measurements

Dataframe solution_df for pickle:



TARGETVAR 
 ... 
 U100_rm3 
 V10_rm3 
 V100_rm3 
 U10_rm3 
 U100_rm3 
 V10_rm3 
 V100_rm3 
 YEAR 
 DAYOFYEAR 
 HOUR 
 
 
 ZONEID 
 1 
 2 
 3 
 4 
 5 
 6 
 7 
 8 
 9 
 10 
 ... 
 9 
 9 
 9 
 10 
 10 
 10 
 10 
 
 
 
 
 
 
 
 0 
 0.000000 
 0.270840 
 0.070839 
 0.032609 
 0.029816 
 0.052044 
 0.140550 
 0.224602 
 0.054787 
 0.121128 
 ... 
 1.473541 
 -3.757118 
 -5.748985 
 -0.165117 
 0.320843 
 -2.658255 
 -4.277716 
 2013 
 213 
 0 
 
 
 1 
 0.019168 
 0.195737 
 0.081343 
 0.028040 
 0.041659 
 0.051252 
 0.152884 
 0.150140 
 0.184262 
 0.235245 
 ... 
 2.425901 
 -3.923264 
 -5.798490 
 -0.257103 
 -0.014866 
 -3.223966 
 -4.800565 
 2013 
 213 
 1 
 
 
 2 
 0.286627 
 0.207469 
 0.107185 
 0.064430 
 0.052353 
 0.086763 
 0.199029 
 0.151965 
 0.389327 
 0.271003 
 ... 
 2.899162 
 -4.043484 
 -5.885207 
 0.085075 
 0.330257 
 -3.693685 
 -5.259272 
 2013 
 213 
 2 
 
 
 3 
 0.287943 
 0.395594 
 0.157022 
 0.055293 
 0.052712 
 0.103719 
 0.281050 
 0.217666 
 0.503941 
 0.272300 
 ... 
 3.475966 
 -4.194771 
 -5.943312 
 0.764744 
 1.048942 
 -4.490173 
 -6.070175 
 2013 
 213 
 3 
 
 
 4 
 0.352317 
 0.509844 
 0.188443 
 0.143825 
 0.089402 
 0.111278 
 0.307617 
 0.312690 
 0.389198 
 0.331207 
 ... 
 3.356855 
 -4.239347 
 -5.972145 
 1.679930 
 2.239443 
 -4.925310 
 -6.571012 
 2013 
 213 
 4 
 
 
 
 5 rows × 45 columns


Solution data frame saved to data/Solution_pp_2017_11_04_21_27_42.pkl

Dataframe solution_df for feather:



,TARGETVAR_1,TARGETVAR_2,TARGETVAR_3,TARGETVAR_4,TARGETVAR_5,TARGETVAR_6,TARGETVAR_7,TARGETVAR_8,TARGETVAR_9,TARGETVAR_10,...,U100_rm3_9,V10_rm3_9,V100_rm3_9,U10_rm3_10,U100_rm3_10,V10_rm3_10,V100_rm3_10,YEAR,DAYOFYEAR,HOUR
0,0.000000,0.270840,0.070839,0.032609,0.029816,0.052044,0.140550,0.224602,0.054787,0.121128,...,1.473541,-3.757118,-5.748985,-0.165117,0.320843,-2.658255,-4.277716,2013,213,0
1,0.019168,0.195737,0.081343,0.028040,0.041659,0.051252,0.152884,0.150140,0.184262,0.235245,...,2.425901,-3.923264,-5.798490,-0.257103,-0.014866,-3.223966,-4.800565,2013,213,1
2,0.286627,0.207469,0.107185,0.064430,0.052353,0.086763,0.199029,0.151965,0.389327,0.271003,...,2.899162,-4.043484,-5.885207,0.085075,0.330257,-3.693685,-5.259272,2013,213,2
3,0.287943,0.395594,0.157022,0.055293,0.052712,0.103719,0.281050,0.217666,0.503941,0.272300,...,3.475966,-4.194771,-5.943312,0.764744,1.048942,-4.490173,-6.070175,2013,213,3
4,0.352317,0.509844,0.188443,0.143825,0.089402,0.111278,0.307617,0.312690,0.389198,0.331207,...,3.356855,-4.239347,-5.972145,1.679930,2.239443,-4.925310,-6.571012,2013,213,4



Solution data frame saved to data/Solution_pp_2017_11_04_21_27_42.feather
